# [ **GitHub**에서 확인하시면 출력까지 확인하실 수 있습니다. ]
https://github.com/knudatascientists/Team1/blob/main/__%EC%B5%9C%EC%A2%85%EB%AA%A8%EB%8D%B8__/merged.ipynb

---

In [ ]:
# Colab에서 구동시 True로 설정하고 실행

Colab = False

Colab용 구글드라이브 마운트

In [ ]:
if Colab:
    from google.colab import drive

    drive.mount('/content/gdrive')

In [ ]:
# cd '/content/gdrive/MyDrive/Project_Methodology'

## 모듈 불러오기

In [ ]:
# if Colab:
#     !pip install beautifulsoup4
#     !pip install sentence_transformers
#     !pip install transformers
#     !pip install konlpy
#     !pip install selenium
#     !apt-get update
#     !apt install chromium-chromedriver
#     !cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
if Colab:
    from selenium import webdriver
    import time
    from selenium.webdriver.common.by import By

    options = webdriver.ChromeOptions()
    options.add_argument('--headless') # browser를 띄우지 않고 실행하기
    options.add_argument('--no-sandbox') # sandbox 기능을 비활성화 하기
    options.add_argument('--disable-dev-shm-usage') # dev/shm/ 폴더를 사용하지 않기

In [ ]:
# 모듈 불러오기
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm

# sklearn
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold, train_test_split

from urllib.request import urlopen #url 주소 호출 라이브러리
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

# tensorflow
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils

# nlp
from konlpy.tag import Kkma, Komoran, Okt
from nltk.corpus import stopwords
import nltk as nlp

from keras.models import load_model

import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel, TFBertModel



import warnings 
warnings.filterwarnings(action='ignore')

c:\ProgramData\Miniconda3\envs\EV_PY39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---  
---  
---

## 주제 분류 모델 첫번째

In [ ]:
data = pd.read_csv('./data/data.csv')

In [ ]:
def makeTextlist(data):
    stopwords_01 = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
    okt = Okt()
    text_list = []
    for text in tqdm(data['title']):
        text = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", " ", text) # 특수문자 제거
        text = text.strip() # 문자 처음과 끝 공백 제거
        tokens = okt.morphs(text) # 단어 추출
        text = [word for word in text if not word in stopwords_01] # 불용어 처리
        text = "".join(text)
        text = text.replace('  ',' ')
        text_list.append(text) 
    
    data["title"] = text_list 

In [ ]:
makeTextlist(data)

100%|██████████| 63931/63931 [01:09<00:00, 917.99it/s] 


In [ ]:
# 단어 개수 column 추가
data['word_counts'] = [len(i.split(' ')) for i in data["title"]]
data['word_counts'].max()

27

In [ ]:
sent_length = data['word_counts'].max()

In [ ]:
# Under Sampling
minCounts = data['topic_idx'].value_counts(dropna=False).min()

data0 = data[data["topic_idx"] == 0][:minCounts]
data1 = data[data["topic_idx"]== 1][:minCounts]
data2 = data[data["topic_idx"]== 2][:minCounts]
data3 = data[data["topic_idx"]== 3][:minCounts]
data4 = data[data["topic_idx"]== 4][:minCounts]
data5 = data[data["topic_idx"]== 5][:minCounts]
data6 = data[data["topic_idx"]== 6][:minCounts]
data = pd.concat([data0, data1, data2, data3, data4, data5, data6], axis = 0)
data.head()

,title,topic_idx,word_counts
617,지카바러스 규명 초저온전현미경 신약연구 유용,0,5
622,증강현실 알파고 어 포켓몬 고 거센 IT 광풍,0,9
643,AI 월드컵 생생 현장 중계,0,5
654,세돌 알파고 집중력 사람 기긴 어렵다 일문일답종합2보,0,7
660,올해 휴대폰 국내 생산량 2천500만대 10년전 18 4,0,8


In [ ]:
finalData = data['title']
finalData.head()

617           지카바러스 규명 초저온전현미경 신약연구 유용
622         증강현실  알파고 어 포켓몬 고 거센 IT 광풍
643                    AI 월드컵 생생 현장 중계
654      세돌 알파고 집중력 사람 기긴 어렵다 일문일답종합2보
660    올해 휴대폰 국내 생산량 2천500만대 10년전 18 4
Name: title, dtype: object

In [ ]:
# target data 생성
target = data['topic_idx']

In [ ]:
corpus = []
for i in finalData:
    corpus.append(i)

In [ ]:
tokenizer = Tokenizer(num_words=10000)  
tokenizer.fit_on_texts(corpus)

In [ ]:
# 정수 인코딩
encoded_docs = tokenizer.texts_to_sequences(corpus)
encoded_docs[0]

[8923, 1274, 5347]

In [ ]:
# 패딩
embedded_docs=pad_sequences(encoded_docs,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 8923 1274 5347]
 [   0    0    0 ...  451 3587 1297]
 [   0    0    0 ... 5960  567 2384]
 ...
 [   0    0    0 ... 8769  262 1475]
 [   0    0    0 ...  326 1555 1287]
 [   0    0    0 ... 4475  104 2853]]


In [ ]:
# 양방향 LSTM
def create_model():

    model1=Sequential()
    model1.add(Embedding(10000,64,input_length=sent_length))
    model1.add(Bidirectional(LSTM(50)))
    model1.add(Dropout(0.3))
    model1.add(Dense(7,activation='softmax'))
    model1.compile(loss='CategoricalCrossentropy',optimizer='adam',metrics=['accuracy'])
    print(model1.summary())
    return model1

In [ ]:
model1 = create_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 27, 64)            640000    
                                                                 
 bidirectional (Bidirectiona  (None, 100)              46000     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 7)                 707       
                                                                 
Total params: 686,707
Trainable params: 686,707
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# numpy 배열로 변경
X_final=np.array(embedded_docs)
y_final=np.array(target)

In [ ]:
# ont-hot 인코딩
y_final = np_utils.to_categorical(y_final)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                   verbose=1, mode='min', baseline=None, restore_best_weights=True)
cp = ModelCheckpoint("./bidirectional_model.h5" ,save_best_only = True)

In [ ]:
hist = model1.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=10,callbacks=[es,cp],batch_size=100)

Epoch 1/10
303/303 [==============================] - 15s 38ms/step - loss: 1.2660 - accuracy: 0.5402 - val_loss: 0.7549 - val_accuracy: 0.7455
Epoch 2/10
303/303 [==============================] - 13s 44ms/step - loss: 0.5851 - accuracy: 0.8093 - val_loss: 0.6831 - val_accuracy: 0.7737
Epoch 3/10
303/303 [==============================] - 14s 45ms/step - loss: 0.4172 - accuracy: 0.8652 - val_loss: 0.7360 - val_accuracy: 0.7678
Epoch 4/10
303/303 [==============================] - 14s 47ms/step - loss: 0.3236 - accuracy: 0.8967 - val_loss: 0.8158 - val_accuracy: 0.7536
Epoch 5/10
303/303 [==============================] - 18s 61ms/step - loss: 0.2652 - accuracy: 0.9145 - val_loss: 0.8712 - val_accuracy: 0.7512
Epoch 5: early stopping


---  
---  
---  

## 주제 분류 모델 두번째

In [ ]:
PATH = './data/'

In [ ]:
train_data = pd.read_csv(PATH + "data.csv")

In [ ]:
test_data = train_data[51144:]
train_data = train_data[:51144]

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

BATCH_SIZE = 32
NUM_EPOCHS = 1          # EPOCH 수 조정 --------------------------------------------------------------------------------------------------------------------------------------------------------------
VALID_SPLIT = 0.2
MAX_LEN = 44 
DATA_IN_PATH = 'data_in/KOR'
DATA_OUT_PATH = "data_out/KOR"

In [ ]:
tokenizer_bert= BertTokenizer.from_pretrained("bert-base-multilingual-cased", cache_dir='bert_ckpt', do_lower_case=False)

In [ ]:
def bert_tokenizer(stc, MAX_LEN):
    
    encoded_dict = tokenizer_bert.encode_plus(
        text = stc,
        add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
        max_length = MAX_LEN,           # Pad & truncate all sentences.
        pad_to_max_length = True,
        return_attention_mask = True    # Construct attn. masks.
        
    )
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] # And its attention mask (simply differentiates padding from non-padding).
    token_type_id = encoded_dict['token_type_ids']  # differentiate two sentences
    
    return input_id, attention_mask, token_type_id

In [ ]:
input_ids = []
attention_masks = []
token_type_ids = []
train_data_labels = []

for train_sent, train_label in tqdm(zip(train_data["title"], train_data["topic_idx"]), total=len(train_data)):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(train_sent, MAX_LEN)   # 토큰화 및 패딩
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        train_data_labels.append(train_label)

    except Exception as e:
        print(e)
        print(train_sent)
        pass

train_input_ids = np.array(input_ids, dtype=int)
train_attention_masks = np.array(attention_masks, dtype=int)
train_type_ids = np.array(token_type_ids, dtype=int)
train_inputs = (train_input_ids, train_attention_masks, train_type_ids)

train_data_labels = np.asarray(train_data_labels, dtype=np.int32) 

print("# sents: {}, # labels: {}".format(len(train_input_ids), len(train_data_labels)))

100%|██████████| 51144/51144 [00:21<00:00, 2350.99it/s]


# sents: 51144, # labels: 51144


In [ ]:
# MODEL CLASS ------------------------------------------------------------------------------------------------------------------------------------
class TFBertClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBertClassifier, self).__init__()

        self.bert = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(num_class, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range), 
                                                name="classifier")
        
    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        
        #outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1] 
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

# ---------------------------------------------------------------------------------------------------------------------------------------------------

model2 = TFBertClassifier(model_name='bert-base-multilingual-cased',
                                  dir_path='bert_ckpt',
                                  num_class=7)

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
optimizer = tf.keras.optimizers.Adam(3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model2.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
model_name = "BERT_klue/bert-base"


earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=2)

checkpoint_path = os.path.join(DATA_OUT_PATH, model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)


# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

data_out/KOR\BERT_klue/bert-base -- Folder already exists 



In [ ]:
history = model2.fit(train_inputs, train_data_labels, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE,
                    validation_split = VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

1279/1279 [==============================] - ETA: 0s - loss: 0.5617 - accuracy: 0.8100
Epoch 1: val_accuracy improved from -inf to 0.55900, saving model to data_out/KOR\BERT_klue/bert-base\weights.h5
1279/1279 [==============================] - 8485s 7s/step - loss: 0.5617 - accuracy: 0.8100 - val_loss: 1.6473 - val_accuracy: 0.5590


---  
---  
---  

#### 첫번째 모델로 타이틀 분류해주는 함수

In [ ]:
def getSubject01(list_title_result) : 
    # 각각의 기사 제목 마다 분류된 값을 저장할 리스트
    list_Subject01 = []
    # 각각의 기사 제목 리스트에서
    for i in range(0, len(list_title_result)):
        # 토큰화 해주고
        test = tokenizer.texts_to_sequences([list_title_result[i]])
        # 패딩해주고
        text = pad_sequences(test, padding='pre', maxlen = 27)
        # 예측값을 저장
        list_Subject01.append(model1.predict(text).argmax())
    # 저장된 리스트를 반환
    return list_Subject01

#### 두번째 모델로 타이틀 분류해주는 함수

In [ ]:
def getSubject02(list_title_result) : 
    # 각각의 기사 제목 마다 분류된 값을 저장할 리스트
    list_Subject02 = []

    input_ids = []
    attention_masks = []
    token_type_ids = []
    data_labels = []

    # 토큰화 및 패딩
    for sent in tqdm(data["title"]): 
        try:
            input_id, attention_mask, token_type_id = bert_tokenizer(sent, MAX_LEN)

            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            token_type_ids.append(token_type_id)
        except Exception as e:
            print(e)
            print(sent)
            pass

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    type_ids = np.array(token_type_ids, dtype=int)
    inputs = (input_ids, attention_masks, type_ids)

    # 결과값 예측
    results = model2.predict(inputs, batch_size=1024)

    # 예측값을 저장
    for i in range(len(results)):
        list_Subject02.append(np.argmax(results[i]))

    # 저장된 리스트를 반환
    return list_Subject02

#### 내용 요약해주는 함수

In [ ]:
def getKeyword(list_text):
    # 각각의 기사 내용을 요약된 값들을 저장할 리스트
    list_Keyword = []
    # 형태소 분류기 호출
    okt = Okt()
    # 저장된 모델 호출
    model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

    # 각각의 기사 내용을
    for i in range(0, len(list_text)) :
        # 토큰화 하고
        tokenized_doc = okt.pos(list_text[i])
        # 명사들만 구성된 문장을 만든다
        tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

        # 단어 묶음 설정, 하나 혹은 2개의 단어로 구성된 요약 형성
        n_gram_range = (1, 2)

        # countVectorizer
        count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])

        candidates = count.get_feature_names_out()
        doc_embedding = model.encode([list_text[i]])
        candidate_embeddings = model.encode(candidates)

        # 요약된 단어는 3개만 도출
        top_n = 3
        # 요약된 단어간 거리 조절
        distances = cosine_similarity(doc_embedding, candidate_embeddings)
        # 거리에 따른 요약된 단어를 3개만 도출
        keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
        # 도출된 단어 리스트를 전체 리스트에 저장
        list_Keyword.append(keywords)
    # 전체 리스트 반환
    return list_Keyword

#### 출력 함수

In [ ]:
def showResult(list_title, list_url, list_Subject01, list_Subject02, list_Keyword) :
    # 값들이 저장된 딕셔너리 형성
    dic = {0 : 'IT', 1 : '경제', 2 : '사회', 3 : '생활문화', 4 : '세계', 5 : '스포츠', 6 : '정치'}
    # 전체 기사에 대한
    for i in range(0, len(list_title)) :
        print('-----------------------------------------------------------')
        print(f'기사 제목 : {list_title[i]}')   # 제목
        print(f'상세 기사 : {list_url[i]}')     # url
        print(f'#{dic[list_Subject01[i]]}, #{dic[list_Subject02[i]]}, #{list_Keyword[i][0]}, #{list_Keyword[i][1]}, #{list_Keyword[i][2]}') #분류(1), #분류(2), #요약(1), #요약(2), #요약(3)
        print('-----------------------------------------------------------')

#### 뉴스 크롤링 함수

In [ ]:
def getNews(n) :
    # 크롤링한 자료들을 저장할 리스트
    list_url_result = []    # url
    list_title_result = []  # 제목
    list_text_result = []   # 내용

    # 드라이버 호출
    if Colab:                                  
        driver = webdriver.Chrome('chromedriver',options=options)
    else:
        driver = webdriver.Chrome("./chromedriver")    
                          
    url = 'https://news.naver.com/' # url 주소 설정
    driver.get(url) # 해당 주소로 드라이버 실행
    driver.implicitly_wait(3)  # 크롤링 방지를 방지하기 위한 대기 3초...
    html = driver.page_source # html 불러오기
    soup = BeautifulSoup(html, 'html.parser')   # BeautifulSoup으로 정리

    # 페이지에 있는 모든 href 받아오기
    list_url = [] # url 저장할 리스트
    list_a = soup.find_all('a') # a가 들어간 모든 부분에서
    for i in range(0, len(list_a)):
        if len(list_a[i]['href']) >= 1 : # href가 있으면 해당 부분 저장
            list_url.append(list_a[i]['href'])

    driver.close()   # 현재 탭 닫기
    driver.quit()    # 브라우저 닫기

    # 전체 list_url 에서 num 개의 뉴스 url만 따로 저장
    list_url_02 = []
    for j in range(91, (91 + (2 * n)), 2) : # n개만
        list_url_02.append(list_url[j])

    # 따로 저장한 url 에서 해당 기사의 제목, url, 기사 내용 추출
    for k in range(0, len(list_url_02)) :
        url = list_url_02[k]
        html = urlopen(url) # url 주소 html로 저장
        soup = BeautifulSoup(html.read(), 'html.parser') # html 데이터 BeautifulSoup으로 요약
        a = soup.contents[2].text
        t = a.split('\n')

        # 기사 페이지에서 가장 긴 부분(= 기사 내용) 부분 인덱스 값 저장
        list_len = []
        for l in range(0, len(t)):
            list_len.append(len(t[l]))
        num = np.argmax(list_len)

        # 타이틀 받아오기
        title = soup.title.text

        list_url_result.append(url)     # url 저장
        list_title_result.append(title) # 제목 저장
        list_text_result.append(t[num]) # 내용 저장

    ####
    print(list_title_result)
    ####
    
    list_Subject01 = getSubject01(list_title_result)    # getSubject01 함수로 기사별 분류 값들을 저장
    list_Subject02 = getSubject02(list_title_result)    # getSubject02 함수로 기사별 분류 값들을 저장
    list_Keyword = getKeyword(list_text_result)         # getKeyword 함수로 기사별 요약 값들을 저장

    # 기사 제목, 기사 주소, 기사 분류(1), 기사 분류(2), 기사 요약 값들을 출력 함수에 전달
    showResult(list_title_result, list_url_result, list_Subject01, list_Subject02, list_Keyword)

In [ ]:
# 지금 네이버 뉴스에 있는 10개의 기사 크롤링 후 분류 및 요약
getNews(10)

NameError: ignored